In [ ]:
# pip install transformers torch datasets rouge-score nltk py7zr evaluate

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


import torch
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

from sklearn.model_selection import train_test_split

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from torch.utils.data import DataLoader


import evaluate


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("samsum")
dataset = {
    'train': dataset['train'].select(range(1000)),
    'test': dataset['test'].select(range(200)),
    'validation': dataset['validation'].select(range(200))
}

corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    all_input_ids = []
    all_attention_masks = []
    all_labels = []

    for dialogue, summary in zip(examples['dialogue'], examples['summary']):
        sentences = sent_tokenize(dialogue)
        summary_sentences = sent_tokenize(summary)


        dialogue_embeddings = sentence_model.encode(sentences)
        summary_embeddings = sentence_model.encode(summary_sentences)

        for idx, sentence in enumerate(sentences):

            tokenized = tokenizer(
                sentence,
                padding='max_length',
                truncation=True,
                max_length=120
            )


            all_input_ids.append(tokenized['input_ids'])
            all_attention_masks.append(tokenized['attention_mask'])


            similarities = cosine_similarity(
                [dialogue_embeddings[idx]],
                summary_embeddings
            )[0]


            is_in_summary = int(np.max(similarities) > 0.7)
            all_labels.append(is_in_summary)

    return {
        'input_ids': all_input_ids,
        'attention_mask': all_attention_masks,
        'labels': all_labels
    }

In [ ]:
tokenized_datasets = {
    'train': dataset['train'].map(
        preprocess_function,
        batched=True,
        batch_size=8,
        remove_columns=dataset['train'].column_names
    ),
    'test': dataset['test'].map(
        preprocess_function,
        batched=True,
        batch_size=8,
        remove_columns=dataset['test'].column_names
    ),
    'validation': dataset['validation'].map(
        preprocess_function,
        batched=True,
        batch_size=8,
        remove_columns=dataset['validation'].column_names
    )
}

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
train_testvalid = tokenized_datasets['train'].train_test_split(test_size=0.2)
train_dataset = train_testvalid['train']
valid_dataset = train_testvalid['test']

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2 )


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
training_args = TrainingArguments(
    output_dir='./extractive_summary_results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=3,
    remove_unused_columns=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)
    labels = pred.label_ids

    return {
        'accuracy': (predictions == labels).mean()
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.241642,0.917201


TrainOutput(global_step=429, training_loss=0.2613631206236797, metrics={'train_runtime': 199.0822, 'train_samples_per_second': 34.453, 'train_steps_per_second': 2.155, 'total_flos': 422971577042400.0, 'train_loss': 0.2613631206236797, 'epoch': 1.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.24164177477359772, 'eval_accuracy': 0.917201166180758, 'eval_runtime': 11.883, 'eval_samples_per_second': 144.324, 'eval_steps_per_second': 9.089, 'epoch': 1.0}


In [ ]:
def generate_summary(text):
    device = next(model.parameters()).device

    sentences = sent_tokenize(text)

    inputs = tokenizer(
        sentences,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    predictions = predictions.cpu()

    selected_indices = predictions[:, 1].argsort(descending=True)[:2]
    summary_sentences = [sentences[i] for i in selected_indices]

    return ' '.join(summary_sentences)

In [ ]:
test_dialogue = """
John: Hey, how was your vacation in Hawaii?
Mary: It was amazing! The beaches were beautiful and the weather was perfect.
John: Did you try any water sports?
"""

print("Original dialogue:")
print(test_dialogue)
print("\nGenerated summary:")
print(generate_summary(test_dialogue))

Original dialogue:

John: Hey, how was your vacation in Hawaii?
Mary: It was amazing! The beaches were beautiful and the weather was perfect.
John: Did you try any water sports?


Generated summary:

John: Hey, how was your vacation in Hawaii? John: Did you try any water sports?


In [ ]:
# test_examples = dataset['test'].select(range(30))

# for idx, example in enumerate(test_examples):
#     print(f"\n=== Example {idx + 1} ===")
#     print("\nOriginal dialogue:")
#     print(example['dialogue'])
#     print("\nOriginal summary:")
#     print(example['summary'])
#     print("\nGenerated summary:")
#     print(generate_summary(example['dialogue']))
#     print("\n" + "="*50)